```sql
WITH InitialData AS (
    SELECT 
        flight_id,
        -- 출발 도착 날짜
        CAST(SUBSTRING(departure_id, 1, 4) || '-' || SUBSTRING(departure_id, 5, 2) || '-' || SUBSTRING(departure_id, 7, 2) AS DATE) AS DEPARTURE_DATE,
        CAST(SUBSTRING(arrival_id, 1, 4) || '-' || SUBSTRING(arrival_id, 5, 2) || '-' || SUBSTRING(arrival_id, 7, 2) AS DATE) AS ARRIVAL_DATE,
        -- 출발 도착 경유 횟수
        LENGTH(departure_id) - LENGTH(REPLACE(departure_id, '+', '')) AS departure_layover_cnt,
        LENGTH(arrival_id) - LENGTH(REPLACE(arrival_id, '+', '')) AS arrival_layover_cnt,
        date_parse(departure_detail[1].sdt , '%Y%m%d%H%i') AS departure_sdt,
        date_parse(arrival_detail[1].sdt , '%Y%m%d%H%i') AS arrival_sdt,
        departure_detail,
        arrival_detail,
        total_fare,
        partition_1
    FROM 
        "instantrip"."20240430" 
),
DirectFlight AS (
SELECT 
    flight_id,
    DEPARTURE_DATE,
    ARRIVAL_DATE,
    departure_sdt,
    arrival_sdt,
    date_diff('day', DEPARTURE_DATE, ARRIVAL_DATE) AS nights,
    total_fare,
    partition_1
FROM InitialData
WHERE departure_layover_cnt = 0 AND arrival_layover_cnt = 0
),
GroupedFlights AS (
SELECT 
    flight_id,
    DEPARTURE_DATE,
    ARRIVAL_DATE,
    departure_sdt,
    arrival_sdt,
    nights,
    total_fare,
    partition_1,
    -- Grouping departure_sdt into 30-minute intervals
    DATE_TRUNC('minute', departure_sdt) + INTERVAL '15' minute * CAST(FLOOR(EXTRACT(MINUTE FROM departure_sdt) / 15) AS INTEGER) AS dep_sdt_30min
FROM DirectFlight
)

SELECT 
    date_format(dep_sdt_30min, '%H%i') AS dep_sdt_30min_formatted,
    COUNT(*) AS flight_count
FROM GroupedFlights
GROUP BY date_format(dep_sdt_30min, '%H%i')
ORDER BY date_format(dep_sdt_30min, '%H%i');
```